In [2]:
!pip install qdrant_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
   

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
client = QdrantClient("ec2-34-223-50-104.us-west-2.compute.amazonaws.com", port=6333)

In [4]:
client.create_collection(
    collection_name="aveta-testing1",
    vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE),
)

True

In [16]:
client.delete_collection(collection_name="aveta")

True

In [5]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00


In [6]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 5.3 MB/s eta 0:00:00


In [7]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=19eef5ca1b32350bb967bafca26f8ad402c2ab4fd6dd2c5fce49d9bab6eb74f0
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant

loader = PyPDFLoader("video_games - video_games.csv.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(separators=["\\n", "\n\n"], chunk_size=10000, chunk_overlap=0)
docs = text_splitter.split_documents(data)
split_output = []
for document in data:
  #print(document)
  lines = document.page_content.split('\n')
  for line in lines:
    split_output.append(line)
    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    # docs = text_splitter.split_documents(data)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "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"
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [9]:
len(docs)

607

In [ ]:
data

In [11]:
import time
start = time.time()
url = "http://ec2-34-223-50-104.us-west-2.compute.amazonaws.com:6333"
qdrant = Qdrant.from_texts(
    split_output,
    embeddings,
    url=url,
    prefer_grpc=False,
    collection_name="aveta-testing1",
)
end = time.time()
time = end - start
print(time)

2762.4807634353638


In [23]:
qdrant

In [12]:
doc_store = Qdrant(
    client=client, collection_name="aveta-testing1",
    embeddings=embeddings,
)

In [47]:
doc_store

In [15]:
query = "Football Manager Handheld 2010 PSP December 22, 2009 Improved Match Engine - massively update AI making the engine more realistic than ever. A brand new skin - giving the game a fresh and vibrant new look. Improved Board Confidence - a complete overhaul to allow the better visibility on how the board views your performance as well as providing feedback on any decisions made. Improved coach and scout reports - it is now possible to view scout reports for individual players from their profiles. Assistant manager team reports - you can now see what your assistant manager thinks of your team, and any suggested improvements he would make. Unlockable managerial abilities - Acquire new abilities as your reputation improves such as the ability to over-rule requests made by the board"
found_docs = doc_store.similarity_search(query)

In [16]:
found_docs

[Document(page_content='Football Manager Handheld 2010 PSP December 22, 2009 Improved Match Engine - massively update AI making the engine more realistic than ever. A brand new skin - giving the game a fresh and vibrant new look. Improved Board Confidence - a complete overhaul to allow the better visibility on how the board views your performance as well as providing feedback on any decisions made. Improved coach and scout reports - it is now possible to view scout reports for individual players from their profiles. Assistant manager team reports - you can now see what your assistant manager thinks of your team, and any suggested improvements he would make. Unlockable managerial abilities - Acquire new abilities as your reputation improves such as the ability to over-rule requests made by the board. New player comparison screens - compare players within your squad or with future transfer targets. Player testimonial matches - players now receive a testimonial game to reward loyalty and 